In [ ]:
# WRKING PROPERLY AND ENHOUGH

import pandas as pd
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
import pickle
import os

# Selected features for anomaly detection
selected_features = ['Age', 'Dosage', 'Quantity']
medication_col = 'Medication_Name'

# Function to extract medication combinations
def get_medication_combination(data):
    """
    Extract a sorted tuple of medications for each transaction by grouping rows with the same Transaction_ID.
    Assumes a 'Transaction_ID' column exists in the dataset.
    """
    try:
        # Group by Transaction_ID and collect medications as a set for each transaction
        grouped = data.groupby('Transaction_ID')[medication_col].apply(
            lambda meds: tuple(sorted(set(meds.dropna().astype(str).str.strip())))
        )
        return grouped
    except Exception as e:
        print(f"Error processing medication combinations: {e}")
        return pd.Series(dtype=object)


# Train an SVM model for anomaly detection on pharmacy data
def train_svm_model(data, selected_features):
    scaler = StandardScaler()
    features = scaler.fit_transform(data[selected_features])
    model = OneClassSVM(kernel='rbf', nu=0.05, gamma='scale').fit(features)
    return model, scaler

# Test the SVM model and detect anomalies
def test_svm_model_with_combinations(model, scaler, test_data, output_file):
    # Transform selected features
    features = scaler.transform(test_data[selected_features])
    predictions = model.predict(features)
    anomalies = test_data[predictions == -1]
    
    # Get medication combinations grouped by Transaction_ID
    medication_combinations = get_medication_combination(test_data)
    combination_counts = medication_combinations.value_counts()
    rare_combinations = combination_counts[combination_counts < 3].index

    # Add a cause for anomalies based on rare medication combinations
    anomalies['Cause'] = anomalies['Transaction_ID'].map(
        lambda tid: "Abnormal medication combination" 
        if medication_combinations.get(tid, ()) in rare_combinations else "Feature anomaly"
    )

    # Save anomalies to CSV
    anomalies.to_csv(output_file, index=False)
    return anomalies


# Aggregation function: averaging models and scalers
def aggregate_models(models, scalers):
    # Averaging the SVM models - using a weighted average based on support vectors
    # Placeholder logic: A more sophisticated approach is required for actual model averaging
    # For simplicity, we'll average the scalers and use the first model as a placeholder
    
    # Averaging the scalers
    avg_mean = np.mean([scaler.mean_ for scaler in scalers], axis=0)
    avg_scale = np.mean([scaler.scale_ for scaler in scalers], axis=0)
    avg_scaler = StandardScaler()
    avg_scaler.mean_ = avg_mean
    avg_scaler.scale_ = avg_scale
    
    # Placeholder: return the first model (advanced averaging is needed for actual models)
    # A full model averaging might involve custom logic for combining support vectors
    avg_model = models[0]  # Simple placeholder, implement a better aggregation for OneClassSVM if needed
    
    return avg_model, avg_scaler

# Directory to save intermediate and final results
output_dir = "anomaly_results"
os.makedirs(output_dir, exist_ok=True)

# Train models for each pharmacy and save them
pharmacies = []
zones = []
cities = []
national = []

# Load and train for each pharmacy
for city in range(1, 4):  # 3 cities
    city_models = []
    city_scalers = []  # Collect scalers for city aggregation
    for zone in range(1, 4):  # 3 zones per city
        zone_models = []
        zone_scalers = []  # Collect scalers for zone aggregation
        for pharmacy in range(1, 5):  # 4 pharmacies per zone
            pharmacy_name = f"Ph{pharmacy:02d}_Z{zone:02d}_C{city:02d}"
            train_file = f"{pharmacy_name}_train.csv"
            train_data = pd.read_csv(train_file)
            
            # Train the SVM model
            model, scaler = train_svm_model(train_data, selected_features)
            pharmacy_model = {'model': model, 'scaler': scaler}
            zone_models.append(pharmacy_model)
            zone_scalers.append(scaler)  # Collect scaler
            print(f"Trained model for {pharmacy_name}")
            
            # Test the model on the TEST dataset
            test_data = pd.read_csv("TEST.csv")
            test_results_file = os.path.join(output_dir, f"{pharmacy_name}_test_results.csv")
            test_svm_model_with_combinations(model, scaler, test_data, test_results_file)
        
        # Aggregate zone models
        zone_model, zone_scaler = aggregate_models([m['model'] for m in zone_models], zone_scalers)
        zone_models = {'model': zone_model, 'scaler': zone_scaler}
        city_models.append(zone_models)
        city_scalers.append(zone_scaler)  # Collect scalers for city aggregation
        print(f"Aggregated model for Zone {zone} of City {city}")
        
        # Test zone model
        test_results_file = os.path.join(output_dir, f"Zone{zone:02d}_City{city:02d}_test_results.csv")
        test_svm_model_with_combinations(zone_model, zone_scaler, test_data, test_results_file)
    
    # Aggregate city models
    city_model, city_scaler = aggregate_models([m['model'] for m in city_models], city_scalers)
    cities.append({'model': city_model, 'scaler': city_scaler})
    print(f"Aggregated model for City {city}")
    
    # Test city model
    test_results_file = os.path.join(output_dir, f"City{city:02d}_test_results.csv")
    test_svm_model_with_combinations(city_model, city_scaler, test_data, test_results_file)

# Aggregate national model
national_model, national_scaler = aggregate_models([c['model'] for c in cities], [c['scaler'] for c in cities])
national = {'model': national_model, 'scaler': national_scaler}
print("Aggregated National model")

# Test national model
test_results_file = os.path.join(output_dir, f"National_test_results.csv")
test_svm_model_with_combinations(national_model, national_scaler, test_data, test_results_file)

print("All models trained, tested, and results saved.")


In [2]:
# WRKING PROPERLY AND ENHOUGH

import pandas as pd
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
import pickle
import os

# Selected features for anomaly detection
selected_features = ['Age', 'Dosage', 'Quantity']
medication_col = 'Medication_Name'




# Function to extract medication combinations
def get_medication_combination(data):
    """
    Extract a sorted tuple of medications for each transaction by grouping rows with the same Transaction_ID.
    Assumes a 'Transaction_ID' column exists in the dataset.
    """
    try:
        # Group by Transaction_ID and collect medications as a set for each transaction
        grouped = data.groupby('Transaction_ID')[medication_col].apply(
            lambda meds: tuple(sorted(set(meds.dropna().astype(str).str.strip())))
        )
        return grouped
    except Exception as e:
        print(f"Error processing medication combinations: {e}")
        return pd.Series(dtype=object)

# Train an SVM model for anomaly detection on pharmacy data
def train_svm_model(data, selected_features):
    scaler = StandardScaler()
    features = scaler.fit_transform(data[selected_features])
    model = OneClassSVM(kernel='rbf', nu=0.05, gamma='scale').fit(features)
    return model, scaler

# Test the SVM model and detect anomalies
def test_svm_model_with_combinations(model, scaler, test_data, output_file):
    # Transform selected features
    features = scaler.transform(test_data[selected_features])
    predictions = model.predict(features)
    anomalies = test_data[predictions == -1]
    
    # Get medication combinations grouped by Transaction_ID
    medication_combinations = get_medication_combination(test_data)
    
    # Print the medication combinations to debug
    print("Medication Combinations Extracted for Test Data:")
    print(medication_combinations.head())  # Print first few combinations
    
    # Get the frequency of each medication combination
    combination_counts = medication_combinations.value_counts()
    
    # Print the combination counts to debug
    print("Medication Combination Counts:")
    print(combination_counts.head())  # Print first few counts
    
    # Identify rare combinations based on threshold < 3 (or your threshold of choice)
    rare_combinations = combination_counts[combination_counts < 3].index
    
    # Print the rare combinations to debug
    print("Rare Combinations (threshold < 3):")
    print(rare_combinations)  # Print the rare combinations
    
    # Add a cause for anomalies based on rare medication combinations
    anomalies['Cause'] = anomalies['Transaction_ID'].map(
        lambda tid: "Abnormal medication combination" 
        if medication_combinations.get(tid, ()) in rare_combinations else "Feature anomaly"
    )

    # Save anomalies to CSV
    anomalies.to_csv(output_file, index=False)
    
    # Print the anomalies detected to debug
    print(f"Anomalies Detected (Total: {len(anomalies)}):")
    print(anomalies[['Transaction_ID', 'Cause']].head())  # Print a few anomalies detected
    
    return anomalies

# Aggregation function: averaging models and scalers
def aggregate_models(models, scalers):
    # Averaging the SVM models - using a weighted average based on support vectors
    # Placeholder logic: A more sophisticated approach is required for actual model averaging
    # For simplicity, we'll average the scalers and use the first model as a placeholder
    
    # Averaging the scalers
    avg_mean = np.mean([scaler.mean_ for scaler in scalers], axis=0)
    avg_scale = np.mean([scaler.scale_ for scaler in scalers], axis=0)
    avg_scaler = StandardScaler()
    avg_scaler.mean_ = avg_mean
    avg_scaler.scale_ = avg_scale
    
    # Placeholder: return the first model (advanced averaging is needed for actual models)
    # A full model averaging might involve custom logic for combining support vectors
    avg_model = models[0]  # Simple placeholder, implement a better aggregation for OneClassSVM if needed
    
    return avg_model, avg_scaler

# Directory to save intermediate and final results
output_dir = "anomaly_results"
os.makedirs(output_dir, exist_ok=True)

# Train models for each pharmacy and save them
pharmacies = []
zones = []
cities = []
national = []

# Load and train for each pharmacy
for city in range(1, 4):  # 3 cities
    city_models = []
    city_scalers = []  # Collect scalers for city aggregation
    for zone in range(1, 4):  # 3 zones per city
        zone_models = []
        zone_scalers = []  # Collect scalers for zone aggregation
        for pharmacy in range(1, 5):  # 4 pharmacies per zone
            pharmacy_name = f"Ph{pharmacy:02d}_Z{zone:02d}_C{city:02d}"
            train_file = f"{pharmacy_name}_train.csv"
            train_data = pd.read_csv(train_file)
            
            # Train the SVM model
            model, scaler = train_svm_model(train_data, selected_features)
            pharmacy_model = {'model': model, 'scaler': scaler}
            zone_models.append(pharmacy_model)
            zone_scalers.append(scaler)  # Collect scaler
            print(f"Trained model for {pharmacy_name}")
            
            # Test the model on the TEST dataset
            test_data = pd.read_csv("TEST.csv")
            test_results_file = os.path.join(output_dir, f"{pharmacy_name}_test_results.csv")
            test_svm_model_with_combinations(model, scaler, test_data, test_results_file)
        
        # Aggregate zone models
        zone_model, zone_scaler = aggregate_models([m['model'] for m in zone_models], zone_scalers)
        zone_models = {'model': zone_model, 'scaler': zone_scaler}
        city_models.append(zone_models)
        city_scalers.append(zone_scaler)  # Collect scalers for city aggregation
        print(f"Aggregated model for Zone {zone} of City {city}")
        
        # Test zone model
        test_results_file = os.path.join(output_dir, f"Zone{zone:02d}_City{city:02d}_test_results.csv")
        test_svm_model_with_combinations(zone_model, zone_scaler, test_data, test_results_file)
    
    # Aggregate city models
    city_model, city_scaler = aggregate_models([m['model'] for m in city_models], city_scalers)
    cities.append({'model': city_model, 'scaler': city_scaler})
    print(f"Aggregated model for City {city}")
    
    # Test city model
    test_results_file = os.path.join(output_dir, f"City{city:02d}_test_results.csv")
    test_svm_model_with_combinations(city_model, city_scaler, test_data, test_results_file)

# Aggregate national model
national_model, national_scaler = aggregate_models([c['model'] for c in cities], [c['scaler'] for c in cities])
national = {'model': national_model, 'scaler': national_scaler}
print("Aggregated National model")

# Test national model
test_results_file = os.path.join(output_dir, f"National_test_results.csv")
test_svm_model_with_combinations(national_model, national_scaler, test_data, test_results_file)

print("All models trained, tested, and results saved.")


Trained model for Ph01_Z01_C01
Medication Combinations Extracted for Test Data:
Transaction_ID
004f1c8c-2859-4efa-906d-c79aac2326b0    (Medication_1, Medication_18, Medication_28, M...
036d1b01-b286-4e7c-87c9-92c21e365308    (Medication_15, Medication_17, Medication_26, ...
03a78243-f17d-4fd8-8dbf-217ea86d7c09    (Medication_14, Medication_2, Medication_25, M...
07e36cee-3763-4466-a549-acc1a1109cfd    (Medication_10, Medication_24, Medication_3, M...
0bea5373-2519-47c1-bdd9-c1af4c208206                                     (Medication_24,)
Name: Medication_Name, dtype: object
Medication Combination Counts:
Medication_Name
(Medication_21,)    5
(Medication_24,)    4
(Medication_16,)    4
(Medication_5,)     3
(Medication_15,)    3
Name: count, dtype: int64
Rare Combinations (threshold < 3):
Index([                                                                 ('Medication_12',),
                                                          ('Medication_1', 'Medication_5'),
                

C:\Users\Nanotec Ss\AppData\Local\Temp\ipykernel_21020\1285101869.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anomalies['Cause'] = anomalies['Transaction_ID'].map(
C:\Users\Nanotec Ss\AppData\Local\Temp\ipykernel_21020\1285101869.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anomalies['Cause'] = anomalies['Transaction_ID'].map(


Medication Combinations Extracted for Test Data:
Transaction_ID
004f1c8c-2859-4efa-906d-c79aac2326b0    (Medication_1, Medication_18, Medication_28, M...
036d1b01-b286-4e7c-87c9-92c21e365308    (Medication_15, Medication_17, Medication_26, ...
03a78243-f17d-4fd8-8dbf-217ea86d7c09    (Medication_14, Medication_2, Medication_25, M...
07e36cee-3763-4466-a549-acc1a1109cfd    (Medication_10, Medication_24, Medication_3, M...
0bea5373-2519-47c1-bdd9-c1af4c208206                                     (Medication_24,)
Name: Medication_Name, dtype: object
Medication Combination Counts:
Medication_Name
(Medication_21,)    5
(Medication_24,)    4
(Medication_16,)    4
(Medication_5,)     3
(Medication_15,)    3
Name: count, dtype: int64
Rare Combinations (threshold < 3):
Index([                                                                 ('Medication_12',),
                                                          ('Medication_1', 'Medication_5'),
                                               

C:\Users\Nanotec Ss\AppData\Local\Temp\ipykernel_21020\1285101869.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anomalies['Cause'] = anomalies['Transaction_ID'].map(
C:\Users\Nanotec Ss\AppData\Local\Temp\ipykernel_21020\1285101869.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anomalies['Cause'] = anomalies['Transaction_ID'].map(
C:\Users\Nanotec Ss\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.

Anomalies Detected (Total: 28):
                          Transaction_ID                            Cause
3   926c711b-02d9-480d-a497-a772e4d8a35f  Abnormal medication combination
27  33fb0471-26ce-4726-8b51-123d3dcbc253  Abnormal medication combination
50  7af7cfe4-152c-4e18-8db6-844b417c7b66  Abnormal medication combination
61  63d6742b-76b7-411f-9d90-399248216509  Abnormal medication combination
63  63d6742b-76b7-411f-9d90-399248216509  Abnormal medication combination
Aggregated model for Zone 1 of City 1
Medication Combinations Extracted for Test Data:
Transaction_ID
004f1c8c-2859-4efa-906d-c79aac2326b0    (Medication_1, Medication_18, Medication_28, M...
036d1b01-b286-4e7c-87c9-92c21e365308    (Medication_15, Medication_17, Medication_26, ...
03a78243-f17d-4fd8-8dbf-217ea86d7c09    (Medication_14, Medication_2, Medication_25, M...
07e36cee-3763-4466-a549-acc1a1109cfd    (Medication_10, Medication_24, Medication_3, M...
0bea5373-2519-47c1-bdd9-c1af4c208206                          

...

C:\Users\Nanotec Ss\AppData\Local\Temp\ipykernel_21020\1285101869.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anomalies['Cause'] = anomalies['Transaction_ID'].map(
C:\Users\Nanotec Ss\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
C:\Users\Nanotec Ss\AppData\Local\Temp\ipykernel_21020\1285101869.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anomalies['Cause'] = anomalies['Transaction_ID']

Medication Combinations Extracted for Test Data:
Transaction_ID
004f1c8c-2859-4efa-906d-c79aac2326b0    (Medication_1, Medication_18, Medication_28, M...
036d1b01-b286-4e7c-87c9-92c21e365308    (Medication_15, Medication_17, Medication_26, ...
03a78243-f17d-4fd8-8dbf-217ea86d7c09    (Medication_14, Medication_2, Medication_25, M...
07e36cee-3763-4466-a549-acc1a1109cfd    (Medication_10, Medication_24, Medication_3, M...
0bea5373-2519-47c1-bdd9-c1af4c208206                                     (Medication_24,)
Name: Medication_Name, dtype: object
Medication Combination Counts:
Medication_Name
(Medication_21,)    5
(Medication_24,)    4
(Medication_16,)    4
(Medication_5,)     3
(Medication_15,)    3
Name: count, dtype: int64
Rare Combinations (threshold < 3):
Index([                                                                 ('Medication_12',),
                                                          ('Medication_1', 'Medication_5'),
                                               

C:\Users\Nanotec Ss\AppData\Local\Temp\ipykernel_21020\1285101869.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anomalies['Cause'] = anomalies['Transaction_ID'].map(
C:\Users\Nanotec Ss\AppData\Local\Temp\ipykernel_21020\1285101869.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anomalies['Cause'] = anomalies['Transaction_ID'].map(
C:\Users\Nanotec Ss\AppData\Local\Temp\ipykernel_21020\1285101869.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

Medication Combinations Extracted for Test Data:
Transaction_ID
004f1c8c-2859-4efa-906d-c79aac2326b0    (Medication_1, Medication_18, Medication_28, M...
036d1b01-b286-4e7c-87c9-92c21e365308    (Medication_15, Medication_17, Medication_26, ...
03a78243-f17d-4fd8-8dbf-217ea86d7c09    (Medication_14, Medication_2, Medication_25, M...
07e36cee-3763-4466-a549-acc1a1109cfd    (Medication_10, Medication_24, Medication_3, M...
0bea5373-2519-47c1-bdd9-c1af4c208206                                     (Medication_24,)
Name: Medication_Name, dtype: object
Medication Combination Counts:
Medication_Name
(Medication_21,)    5
(Medication_24,)    4
(Medication_16,)    4
(Medication_5,)     3
(Medication_15,)    3
Name: count, dtype: int64
Rare Combinations (threshold < 3):
Index([                                                                 ('Medication_12',),
                                                          ('Medication_1', 'Medication_5'),
                                               

C:\Users\Nanotec Ss\AppData\Local\Temp\ipykernel_21020\1285101869.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anomalies['Cause'] = anomalies['Transaction_ID'].map(
C:\Users\Nanotec Ss\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
C:\Users\Nanotec Ss\AppData\Local\Temp\ipykernel_21020\1285101869.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anomalies['Cause'] = anomalies['Transaction_ID']